In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

In [14]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

user_item_matrix = ratings.pivot(index="userId", columns="movieId", values="rating").fillna(0)

user_similarity = cosine_similarity(user_item_matrix)

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_item_matrix.index)}

In [15]:
def get_recommendations(target_user_id, K, top_N=10):
    target_user_idx = user_to_idx.get(target_user_id, None)

    if target_user_idx is None:
        return []
    
    sim_scores = list(enumerate(user_similarity[target_user_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:]

    top_K_users = [user for user, _ in sim_scores[:K]]
    
    recommendations = []

    for user in top_K_users:
        movies_rated_by_user = user_item_matrix.loc[user_item_matrix.index == user].values
        target_user_ratings = user_item_matrix.loc[user_item_matrix.index == target_user_id].values
       
        unrated_movies = np.where(target_user_ratings == 0, movies_rated_by_user, 0)

        movie_indices = np.argsort(unrated_movies[0])[::-1][:top_N]

        recommended_movies = user_item_matrix.columns[movie_indices]
        recommendations.extend(recommended_movies)

    return recommendations[:top_N]

In [16]:
user_input = widgets.IntText(value=1, description = "User ID:")
K_input = widgets.IntSlider(min=1, max=50, step=1, description = "K:")
top_N_input = widgets.IntSlider(min=1, max=50,step=1, description="Top N:")

In [17]:
def update_recommendations(change):
    target_user_id = user_input.value
    K = K_input.value
    top_N = top_N_input.value

    recommendations = get_recommendations(target_user_id, K, top_N)

    output.clear_output()
    with output:
        if recommendations:
            print(f"Top {top_N} Recommended Movies for User {target_user_id}:")
            for i, movie_id in enumerate(recommendations):
                movie_title = movies[movies["movieId"] == movie_id]["title"].values[0]
                print(f"{i+1}. {movie_title}")
        else:
            print("User not found or not enough data for recommendations.")

In [18]:
search_button = widgets.Button(description = "Get Recommendations")
search_button.on_click(update_recommendations)

output = widgets.Output()
layout = widgets.VBox([user_input, K_input, top_N_input, search_button, output])

display(layout)